# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [101]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
csvfile = "../WeatherPy/citiesoutput.csv"
cities_df = pd.read_csv(csvfile)
cities_df

,Unnamed: 0,Name,Country,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Emerald,AU,148.1667,-23.5333,67.93,72.0,4.0,7.20,1.630188e+09
1,1,Busselton,AU,115.3333,-33.6500,54.09,84.0,48.0,13.00,1.630188e+09
2,2,Tiarei,PF,-149.3333,-17.5333,80.62,81.0,24.0,11.59,1.630188e+09
3,4,Thompson,CA,-97.8558,55.7435,69.73,46.0,100.0,5.88,1.630188e+09
4,5,Bokoro,TD,17.0568,12.3807,81.81,68.0,41.0,7.52,1.630188e+09
...,...,...,...,...,...,...,...,...,...,...
556,608,Baracoa,CU,-74.4958,20.3467,83.62,75.0,95.0,5.48,1.630189e+09
557,609,Whitehorse,CA,-135.0538,60.7161,50.02,94.0,100.0,2.13,1.630189e+09
558,610,Platteville,US,-90.4785,42.7342,93.47,62.0,40.0,12.66,1.630189e+09
559,611,Nizhniy Kuranakh,RU,125.4869,58.8353,40.87,77.0,3.0,1.32,1.630189e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

#create locations
locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df['Humidity']

In [29]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'}

gmaps.figure(layout=figure_layout)
    
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
#filter out cities that do not have a max temp bt 77 and 87 and wind speed 10 or less
filter = cities_df.loc[(cities_df['Max Temp'] > 87) | (cities_df['Max Temp'] < 77) | (cities_df['Wind Speed'] > 10) ]
hotel_df = cities_df.drop(filter.index)

#drop useless column 
hotel_df.drop(['Unnamed: 0'], axis= 1, inplace= True)

#display
hotel_df

,Name,Country,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
4,Bokoro,TD,17.0568,12.3807,81.81,68.0,41.0,7.52,1.630188e+09
10,Jamestown,US,-79.2353,42.0970,85.84,69.0,1.0,5.99,1.630188e+09
14,George Town,MY,100.3354,5.4112,80.53,87.0,20.0,3.44,1.630188e+09
39,Praia,CV,-23.5087,14.9215,77.54,83.0,20.0,4.61,1.630188e+09
43,Sur,OM,59.5289,22.5667,82.04,68.0,4.0,9.98,1.630188e+09
...,...,...,...,...,...,...,...,...,...
544,Kimbe,PG,150.1429,-5.5502,79.54,82.0,28.0,3.85,1.630189e+09
546,Buala,SB,159.5921,-8.1450,82.13,77.0,100.0,9.31,1.630189e+09
554,Pochutla,MX,-96.4661,15.7432,83.82,72.0,100.0,6.78,1.630189e+09
556,Baracoa,CU,-74.4958,20.3467,83.62,75.0,95.0,5.48,1.630189e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df['Hotel Name'] = ' '
hotel_df

,Name,Country,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
4,Bokoro,TD,17.0568,12.3807,81.81,68.0,41.0,7.52,1.630188e+09,
10,Jamestown,US,-79.2353,42.0970,85.84,69.0,1.0,5.99,1.630188e+09,
14,George Town,MY,100.3354,5.4112,80.53,87.0,20.0,3.44,1.630188e+09,
39,Praia,CV,-23.5087,14.9215,77.54,83.0,20.0,4.61,1.630188e+09,
43,Sur,OM,59.5289,22.5667,82.04,68.0,4.0,9.98,1.630188e+09,
...,...,...,...,...,...,...,...,...,...,...
544,Kimbe,PG,150.1429,-5.5502,79.54,82.0,28.0,3.85,1.630189e+09,
546,Buala,SB,159.5921,-8.1450,82.13,77.0,100.0,9.31,1.630189e+09,
554,Pochutla,MX,-96.4661,15.7432,83.82,72.0,100.0,6.78,1.630189e+09,
556,Baracoa,CU,-74.4958,20.3467,83.62,75.0,95.0,5.48,1.630189e+09,


In [149]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "prominence",
    "type": "lodging",
    "radius": 5000,
    "key": g_key}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get city name from DF
    city = row['Name']
    # add  to params dict
    params["location"]= f"{row['Latitude']},{row['Longitude']}"
    # assemble url and make API request
    print("Retrieving Results for city " + row['Name'])
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = 'NA'

Retrieving Results for city Bokoro
Retrieving Results for city Jamestown
Retrieving Results for city George Town
Retrieving Results for city Praia
Retrieving Results for city Sur
Retrieving Results for city Gwadar
Retrieving Results for city Acapulco de Juárez
Retrieving Results for city Kapaa
Retrieving Results for city Mangai
Retrieving Results for city Cacheu
Retrieving Results for city Americus
Retrieving Results for city Lahaina
Retrieving Results for city Patnāgarh
Retrieving Results for city São Paulo de Olivença
Retrieving Results for city Constantine
Retrieving Results for city Kwang Binh
Retrieving Results for city Nushki
Retrieving Results for city Sulangan
Retrieving Results for city Kalaīkunda
Retrieving Results for city Shelburne
Retrieving Results for city Beyneu
Retrieving Results for city Jhajjar
Retrieving Results for city Amalāpuram
Retrieving Results for city Kieta
Retrieving Results for city Urucará
Retrieving Results for city Altoona
Retrieving Results for city Va

In [151]:

hotel_df

,Name,Country,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
4,Bokoro,TD,17.0568,12.3807,81.81,68.0,41.0,7.52,1.630188e+09,NA
10,Jamestown,US,-79.2353,42.0970,85.84,69.0,1.0,5.99,1.630188e+09,Hampton Inn & Suites Jamestown
14,George Town,MY,100.3354,5.4112,80.53,87.0,20.0,3.44,1.630188e+09,Cititel Penang
39,Praia,CV,-23.5087,14.9215,77.54,83.0,20.0,4.61,1.630188e+09,LT Aparthotel
43,Sur,OM,59.5289,22.5667,82.04,68.0,4.0,9.98,1.630188e+09,Sur Plaza Hotel
...,...,...,...,...,...,...,...,...,...,...
544,Kimbe,PG,150.1429,-5.5502,79.54,82.0,28.0,3.85,1.630189e+09,Kimbe Bay Hotel
546,Buala,SB,159.5921,-8.1450,82.13,77.0,100.0,9.31,1.630189e+09,Maringe Lagoon Lodge
554,Pochutla,MX,-96.4661,15.7432,83.82,72.0,100.0,6.78,1.630189e+09,Hotel Posada San Jose
556,Baracoa,CU,-74.4958,20.3467,83.62,75.0,95.0,5.48,1.630189e+09,Hostal Rio Miel


{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}


,Name,Country,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
10,Jamestown,US,-79.2353,42.097,85.84,69.0,1.0,5.99,1.630188e+09,NA


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
